**Proyecto - Simulación Numérica de Fluidos con las Ecuaciones de Euler en 1D (Método de Lax-Friedrich)**

***Integrantes:***

*   María Fernanda Estupiñan Aguilar - 2210727
*   Jefferson Meza - 2231482
*   Fabián Villamizar - 2130438
*   Cristian Ariza - 2221968

Link del Póster: [AQUÍ](https://www.canva.com/design/DAG4Y25CdSQ/KiLhed5YPqkiwdQ1G8HXAw/edit?utm_content=DAG4Y25CdSQ&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton) 🌈

In [ ]:
#Librerías a Usar
import numpy as np
from numba import cuda, float32
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time

In [ ]:
#Parámetros Físicos y numéricos
N = 100
gamma = 1.4
pasos = int(input("¿Cuántos pasos deseas simular? (Ej: 100): "))
dx = 1.0 / N
dt = 0.001

In [ ]:
#Inicio tipo sod
U = np.zeros((N, 3), dtype=np.float32)
for i in range(N):
    if i < N // 2:
        rho, p, u = 1.0, 1.0, 0.0
    else:
        rho, p, u = 0.125, 0.1, 0.0
    E = p / (gamma - 1) + 0.5 * rho * u**2
    U[i] = [rho, rho * u, E]

historia_rho = [U[:, 0].copy()]

In [ ]:
#Kernel CUDA (Se corrió el código con una tarjeta gráfica NVIDIA GeForce GTX 1650 Ti)
@cuda.jit
def calcular_flujo(U, F, gamma):
    i = cuda.grid(1)
    if i < U.shape[0]:
        rho = U[i, 0]
        mom = U[i, 1]
        E = U[i, 2]
        u = mom / rho
        p = (gamma - 1) * (E - 0.5 * rho * u**2)

        F[i, 0] = mom
        F[i, 1] = mom * u + p
        F[i, 2] = u * (E + p)

In [ ]:
#Paso de tiempo (Lax-Friedrichs)
@cuda.jit
def paso_tiempo(U, F, U_new, dx, dt):
    i = cuda.grid(1)
    if 1 <= i < U.shape[0] - 1:
        for j in range(3):
            U_new[i, j] = 0.5 * (U[i + 1, j] + U[i - 1, j]) - dt / (2 * dx) * (F[i + 1, j] - F[i - 1, j])
    if i == 0 or i == U.shape[0] - 1:
        for j in range(3):
            U_new[i, j] = U[i, j]

In [ ]:
#Cálculos Simulación
d_U = cuda.to_device(U)
d_F = cuda.device_array_like(U)
d_U_new = cuda.device_array_like(U)

threads = 64
blocks = (N + threads - 1) // threads

start = time.time()

for _ in range(pasos):
    calcular_flujo[blocks, threads](d_U, d_F, gamma)
    paso_tiempo[blocks, threads](d_U, d_F, d_U_new, dx, dt)
    d_U.copy_to_device(d_U_new)
    historia_rho.append(d_U.copy_to_host()[:, 0])

end = time.time()
print(f"⏱️ Tiempo CUDA: {end - start:.4f} segundos")

In [ ]:
#Animación
fig, ax = plt.subplots()
line, = ax.plot(historia_rho[0], lw=2)
ax.set_ylim(0, 1.1)
ax.set_title("Euler 1D - CUDA")

def actualizar(frame):
    line.set_ydata(historia_rho[frame])
    ax.set_xlabel(f"Paso: {frame}")
    return line,

ani = animation.FuncAnimation(fig, actualizar, frames=len(historia_rho), interval=30)
plt.show()